In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString

Joining Datasets

The goal of training models for classifying routes on the points of interest they pass by in sight, we had first to match our cleaned routes data with our cleaned points of interest (poi) data. We chose a spacial join from the geopandas library for matching on the geometry of both datasets. 

Of cause, this being a study project, we did this for only 451 scraped and cleaned routes from wandermap.net and a (satisfactory large) subset of 18 types of poi scraped from openstreetmap.org. 

In [2]:
#reading in the routes and poi data
routes = pd.read_csv('cleaned_routes_data/cleaned_all_routes_data_long.csv')
#poi = pd.read_csv('cleaned_all_poi_data.csv').iloc[0:5000, :] #using smaller sample for experimenting
poi = pd.read_csv('cleaned_all_poi_data.csv')

In [3]:
#converting routes df into a GeoDataFrame
gdf_routes = gpd.GeoDataFrame(routes, geometry=gpd.points_from_xy(routes.latitude, routes.longitude))
gdf_routes.drop(['Unnamed: 0', 'lat_lgt'], axis=1, inplace=True)
gdf_routes

,route_id,num_of_waypoint,latitude,longitude,geometry
0,1005019,0,52.50607,13.33208,POINT (52.50607 13.33208)
1,1005019,1,52.50553,13.33163,POINT (52.50553 13.33163)
2,1005019,2,52.50525,13.33148,POINT (52.50525 13.33148)
3,1005019,3,52.50515,13.33337,POINT (52.50515 13.33337)
4,1005019,4,52.50520,13.33366,POINT (52.50520 13.33366)
...,...,...,...,...,...
186229,933359,151,52.50444,13.38246,POINT (52.50444 13.38246)
186230,933359,152,52.50525,13.38633,POINT (52.50525 13.38633)
186231,933359,153,52.50643,13.38615,POINT (52.50643 13.38615)
186232,933359,154,52.50648,13.39023,POINT (52.50648 13.39023)


In [4]:
#converting points from routes data to linestring and adding these into a new column
gdf_routes['route_linestring'] = gdf_routes['route_id'].map(gdf_routes.groupby(['route_id'])['geometry'].apply(lambda x: LineString(x.tolist())))
gdf_routes

,route_id,num_of_waypoint,latitude,longitude,geometry,route_linestring
0,1005019,0,52.50607,13.33208,POINT (52.50607 13.33208),"LINESTRING (52.50607 13.33208, 52.50553 13.331..."
1,1005019,1,52.50553,13.33163,POINT (52.50553 13.33163),"LINESTRING (52.50607 13.33208, 52.50553 13.331..."
2,1005019,2,52.50525,13.33148,POINT (52.50525 13.33148),"LINESTRING (52.50607 13.33208, 52.50553 13.331..."
3,1005019,3,52.50515,13.33337,POINT (52.50515 13.33337),"LINESTRING (52.50607 13.33208, 52.50553 13.331..."
4,1005019,4,52.50520,13.33366,POINT (52.50520 13.33366),"LINESTRING (52.50607 13.33208, 52.50553 13.331..."
...,...,...,...,...,...,...
186229,933359,151,52.50444,13.38246,POINT (52.50444 13.38246),"LINESTRING (52.50157 13.49061, 52.50300 13.484..."
186230,933359,152,52.50525,13.38633,POINT (52.50525 13.38633),"LINESTRING (52.50157 13.49061, 52.50300 13.484..."
186231,933359,153,52.50643,13.38615,POINT (52.50643 13.38615),"LINESTRING (52.50157 13.49061, 52.50300 13.484..."
186232,933359,154,52.50648,13.39023,POINT (52.50648 13.39023),"LINESTRING (52.50157 13.49061, 52.50300 13.484..."


In [5]:
#converting poi df into a GeoDataFrame
gdf_poi = gpd.GeoDataFrame(poi, geometry=gpd.points_from_xy(poi.lat, poi.lon))
gdf_poi

,Unnamed: 0,category,name,id,lat,lon,geometry
0,0,['atm'],Bank für Sozialwirtschaft,78252154,52.523744,13.398627,POINT (52.52374 13.39863)
1,1,['atm'],Sparda-Bank,87036263,52.532985,13.384282,POINT (52.53299 13.38428)
2,2,['atm'],Bankhaus August Lenz,89275133,52.518025,13.406956,POINT (52.51802 13.40696)
3,3,['atm'],NaN,213106623,52.542170,13.441137,POINT (52.54217 13.44114)
4,4,['atm'],Berliner Sparkasse,213113204,52.542750,13.392862,POINT (52.54275 13.39286)
...,...,...,...,...,...,...,...
213006,213006,['viewpoint'],NaN,8931299152,52.487989,13.275393,POINT (52.48799 13.27539)
213007,213007,['viewpoint'],NaN,9024702237,52.506772,13.334563,POINT (52.50677 13.33456)
213008,213008,['viewpoint'],Alpengipfel,9026936271,52.401704,13.366960,POINT (52.40170 13.36696)
213009,213009,['viewpoint'],NaN,9038673666,52.482133,13.291911,POINT (52.48213 13.29191)


In [6]:
#spacial joining both datasets on nearest distance of any a route's waypoint from a poi
poi_routes = gpd.sjoin_nearest(gdf_poi, gdf_routes, how='left', max_distance=0.001, distance_col='distance')

#followed by some manipulation for the optics
poi_routes.drop(['Unnamed: 0', 'id', 'lat', 'lon', 'index_right', 'latitude', 'longitude'], axis=1, inplace=True) #drop info to reduce risk of overfitting
poi_routes = poi_routes.iloc[:, [3,5,4,2,0,1,6]] #rearranging columns
poi_routes.rename({'geometry': 'poi_lat_lgt', 'category': 'poi_category', 'name': 'poi_name'}, axis=1, inplace=True) #renaming cloumns
poi_routes['route_id'] = poi_routes['route_id'].astype(int, errors='ignore') #converting float to int for some reason isn't working
poi_routes.dropna(thresh=5, inplace=True) #dropping any poi without a route passing by in max_distance
poi_routes.sort_values(by=['route_id', 'num_of_waypoint'], inplace=True) #sorting by route_id and by number of waypoint to keep the order
poi_routes.reset_index(drop=True, inplace=True) #reset index
poi_routes


,route_id,route_linestring,num_of_waypoint,poi_lat_lgt,poi_category,poi_name,distance
0,113043.0,"LINESTRING (52.45147 13.69072, 52.45147 13.690...",133.0,POINT (52.43852 13.67747),['viewpoint'],Müggeleck,0.000138
1,113043.0,"LINESTRING (52.45147 13.69072, 52.45147 13.690...",282.0,POINT (52.44431 13.62791),['bench'],NaN,0.000551
2,113043.0,"LINESTRING (52.45147 13.69072, 52.45147 13.690...",288.0,POINT (52.44445 13.62716),['bench'],NaN,0.000122
3,113043.0,"LINESTRING (52.45147 13.69072, 52.45147 13.690...",288.0,POINT (52.44448 13.62695),['bench'],NaN,0.000110
4,113043.0,"LINESTRING (52.45147 13.69072, 52.45147 13.690...",292.0,POINT (52.44532 13.62739),['bench'],NaN,0.000319
...,...,...,...,...,...,...,...
91099,3674009.0,"LINESTRING (52.53883 13.21284, 52.53883 13.212...",388.0,POINT (52.58956 13.28346),['tree'],NaN,0.000263
91100,3674009.0,"LINESTRING (52.53883 13.21284, 52.53883 13.212...",388.0,POINT (52.58903 13.28439),['tree'],NaN,0.000801
91101,3674009.0,"LINESTRING (52.53883 13.21284, 52.53883 13.212...",388.0,POINT (52.58895 13.28449),['tree'],NaN,0.000932
91102,3674128.0,"LINESTRING (52.48880 13.26176, 52.48877 13.261...",281.0,POINT (52.45165 13.19131),['tree'],NaN,0.000239


In [7]:
#writing joint sample data into a csv file
#poi_routes.to_csv('joint_sample_data.csv')

#writing joint data into a csv file
poi_routes.to_csv('joint_data.csv')

Notes on joining routes and points of interest data

For discovering at which points of interest our touristic and hiking routes pass by (which we believe justifies pedestrians to chose those routes in the first place) we had to join our routes data and our data about specific points of interest.

The geopandas method sjoin_nearest matches the coordinates of every point of interest (poi) to the nearest waypoints from our routes dataset, if they are closer than the defined max_distance. We had grouped the waypoints before and had connected a linestring for the whole route to every waypoint, so that after joining the poi are essentially matched with information on full routes.

By left joining poi and routes, we ensured that we could detect several poi per route. That gave us far more matches than joining on the linestring geometry from the routes data. 

If there are several route's waypoints equally near to a poi, sjoin_nearest matches both.

Since all calculations were only applied to a small area (Berlin), coordinate reference system (CRS) metrics for the max_distance parameter were neglectable. We found a tolerable distance value (max_distance=0.001) by (visual) experimentation. That distance from the route translates into the poi being at the other side of the street, or elseway visible to a pedestrian.